# PFN Internship 2022 "JE06. Matlantis向けの物性値計算アルゴリズムの開発" レポート課題

以下の最適化問題について考えてもらいます。この問題はPythonで書かれています。

なお、もし本課題よりも"JE19. 材料に関する機械学習や原子シミュレーションの開発・応用研究"の課題のほうが取り組みやすいと感じた場合は、そちらの課題への解答で代替することも認めます。

## 問題設定

3次元空間中に64個のラベルづけされた点を配置します。点の位置は重なることはないものとします。ラベルは整数で、0から7までの値を持つ点がそれぞれ8個ずつあります。この点の集合に対してスカラー値を返すような関数を考えます。このとき、返り値がなるべく小さくなるような点の配置を考えてください。

この課題は、原子構造を推定する問題を模しています。点を原子、スカラー値をエネルギーとみなすことで、エネルギーが低く安定となるような原子配置を考える問題とみなすことができます。

## 関数について

関数の具体的な形を以下に`LennardJonesI22`クラスの`calculate`関数として提供します。詳細については後述しますが、ここではブラックボックス的な関数とみなしてしまっても構いません。

引数として、点のラベルに相当する`atom_type` (64次元、整数)と点の座標に相当する`positions` (64x3次元、浮動小数点数)をとります。出力にはスカラー値の`output`と、`output`の座標に対する微分値の`grad`の組を出力します。微分値は特に使わなくても結構です。

これはいわゆるLennard-Jonesポテンシャルとして知られているものです。すべての点の組み合わせに対して、2点間の距離と2つの点のラベルを引数として定義される関数(2体間関数)の返り値の和となっています。

2体間関数は、距離が十分離れているときは0に漸近します。距離が一定まで近づく範囲では値が下がり、それ以上近づくと値が急激に上昇する振る舞いをします。そのため、それぞれの点をある程度の安定距離をとって配置するのが良い解になります。安定距離は2から4の間くらいです。


In [1]:
from typing import Tuple

import numpy as np


class LennardJonesI22:
    def __init__(self):
        sigma_single = np.array([2.0 + 0.2 * x for x in range(8)])
        epsilon_single = np.array([0.1 + 0.1 * x for x in range(8)])

        self.sigma_matrix = 0.5 * (sigma_single[None, :] + sigma_single[:, None])
        self.epsilon_matrix = np.sqrt(epsilon_single[None, :] * epsilon_single[:, None])

    def calculate(self, atom_type: np.ndarray, positions: np.ndarray) -> Tuple[float, np.ndarray]:
        assert len(positions.shape) == 2
        assert positions.shape[1] == 3
        n_atoms = positions.shape[0]
        
        assert len(atom_type.shape) == 1
        assert atom_type.shape[0] == n_atoms

        an_axis0 = np.repeat(atom_type[None, :], n_atoms, axis=0)
        an_axis1 = np.repeat(atom_type[:, None], n_atoms, axis=1)
        sigma_pairs = self.sigma_matrix[an_axis0, an_axis1]
        epsilon_pairs = self.epsilon_matrix[an_axis0, an_axis1]
    
        x1 = positions[None, :, :]
        x2 = positions[:, None, :]
        x_diff = x2 - x1
        rsq = np.sum(np.square(x_diff), axis=2)
        rsq_reciprocal = np.reciprocal(rsq, out=np.zeros_like(rsq), where=(rsq != 0.0))

        sigma_by_r2 = np.square(sigma_pairs) * rsq_reciprocal
        sigma_by_r6 = np.power(sigma_by_r2, 3)
        sigma_by_r12 = np.square(sigma_by_r6)
        e_pairs = 4.0 * epsilon_pairs * (sigma_by_r12 - sigma_by_r6)
        f_pairs_by_r = -24.0 * epsilon_pairs * (2 * sigma_by_r12 - sigma_by_r6) * rsq_reciprocal
        f_pairs = f_pairs_by_r[:, :, None] * x_diff
    
        output_atoms = 0.5 * np.sum(e_pairs, axis=1)
        output = float(np.sum(output_atoms).item())
        if np.any(rsq + np.identity(rsq.shape[0]) == 0.0):  # Same position
            output = float("inf")
        grad = 0.5 * (np.sum(f_pairs, axis=1) - np.sum(f_pairs, axis=0))
        
        return output, grad

これを使って、適当な入力に対してスカラー値(エネルギー)を計算することができます。以下は3次元空間に一直線に並べたときの計算例です。

In [2]:
atom_type = np.array([0, 1, 2, 3, 4, 5, 6, 7] * 8)
positions = np.array([[3.5 * x, 0.0, 0.0] for x in range(64)])

f = LennardJonesI22()
f_out, _ = f.calculate(atom_type, positions)
print("output: {:.5f}".format(f_out))

print("atom_type")
print(atom_type.tolist())
print("positions")
print(positions.tolist())

output: -20.32051
atom_type
[0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7]
positions
[[0.0, 0.0, 0.0], [3.5, 0.0, 0.0], [7.0, 0.0, 0.0], [10.5, 0.0, 0.0], [14.0, 0.0, 0.0], [17.5, 0.0, 0.0], [21.0, 0.0, 0.0], [24.5, 0.0, 0.0], [28.0, 0.0, 0.0], [31.5, 0.0, 0.0], [35.0, 0.0, 0.0], [38.5, 0.0, 0.0], [42.0, 0.0, 0.0], [45.5, 0.0, 0.0], [49.0, 0.0, 0.0], [52.5, 0.0, 0.0], [56.0, 0.0, 0.0], [59.5, 0.0, 0.0], [63.0, 0.0, 0.0], [66.5, 0.0, 0.0], [70.0, 0.0, 0.0], [73.5, 0.0, 0.0], [77.0, 0.0, 0.0], [80.5, 0.0, 0.0], [84.0, 0.0, 0.0], [87.5, 0.0, 0.0], [91.0, 0.0, 0.0], [94.5, 0.0, 0.0], [98.0, 0.0, 0.0], [101.5, 0.0, 0.0], [105.0, 0.0, 0.0], [108.5, 0.0, 0.0], [112.0, 0.0, 0.0], [115.5, 0.0, 0.0], [119.0, 0.0, 0.0], [122.5, 0.0, 0.0], [126.0, 0.0, 0.0], [129.5, 0.0, 0.0], [133.0, 0.0, 0.0], [136.5, 0.0, 0.0], [140.0, 0.0, 0.0], [143.5, 0.0, 0.0], [1

## 解答方法

結果(入力の組とそのときのスコア)とどのような考え方をしたかを記したレポート(PDFでA4用紙1枚以下程度、1枚に書き切れない場合は多少オーバーしても構いません)と実行に使ったプログラムのソースコードの2点を提出してください。プログラムは配布のJupyter Notebookをそのまま使っても問題ありませんが、別の形式でも構いません。

この課題では、長い時間をかけてスコアをぎりぎりまで詰める必要はありません。その代わり、見通しのよいプログラムを心がけてください。また、レポートではどのようなことを考えて最適化を行ったのかを書くようにしてください。

## 補足: 解き方について

解き方は自由です。微分値があることから様々な勾配法、あるいはPyTorchなどのニューラルネットワークライブラリの最適化関数などを使ってもよいですし、ブラックボックス最適化問題とみなしてOptunaなどのブラックボックス最適化ライブラリを使うのも構いません。

また、原子シミュレーションの知識のある人は、分子動力学計算における各種最適化手法、MCMC法や焼きなまし法などを使って解いても構いません。以下にPythonの原子シミュレーションライブラリASEを使った計算例を示します。(環境へのASEのインストール(`pip install ase`など)が必要です。)


In [3]:
import numpy as np
import ase
import ase.optimize
from ase.calculators.calculator import Calculator, all_changes


class LennardJonesI22Calculator(Calculator):
    implemented_properties = ['energy', 'forces', 'free_energy']
    
    def __init__(self, **kwargs):
        Calculator.__init__(self, **kwargs)
        self.lj_core = LennardJonesI22()
        
    def calculate(self, atoms=None, properties=None, system_changes=all_changes):
        if properties is None:
            properties = self.implemented_properties

        Calculator.calculate(self, atoms, properties, system_changes)
        e_total, f_atoms = self.lj_core.calculate(atoms.get_atomic_numbers(), atoms.get_positions())
        self.results["energy"] = e_total
        self.results["free_energy"] = e_total
        self.results["forces"] = -f_atoms
        

def run_ase():
    calculator = LennardJonesI22Calculator()

    atomic_numbers = np.array([0, 1, 2, 3, 4, 5, 6, 7] * 8)
    positions = np.array([[3.5 * x, 0.0, 0.0] for x in range(len(atomic_numbers))])

    atoms = ase.Atoms(numbers=atomic_numbers, positions=positions)
    atoms.calc = calculator

    opt = ase.optimize.BFGS(atoms, logfile=None)
    opt.run(fmax=0.001)
    
    print("output: {:.5f}".format(atoms.get_potential_energy()))

    
run_ase()

output: -26.69947


## 補足: 関数の詳細な形

以下は問題を解くのに必ずしも必要のない情報ですが、中身を知りたい人向けの説明です。もし以下の説明文に間違いがあった場合、実装のほうを正解としてください。

Lennard-Jonesポテンシャルは、原子間距離をrとしたときに以下の形で表現される2体間ポテンシャルです。

$$
U(r)=4\epsilon \left\{\left(\frac{\sigma}{r}\right)^{12}-\left(\frac{\sigma}{r}\right)^{6}\right\}
$$

εとσは定数です。かっこ内の第1項が距離の-12乗に比例する反発力、第2項が距離の-6乗に比例する吸引力となっています。εは結合の強さを表す定数、σは距離のスケールを表す定数です。

関数の振る舞いとして、rが0に漸近すると無限大に発散し、rの無限遠では0に漸近します。微分するとわかりますが、おおよそrがσの1.12倍程度のところで最小値をとります。

εとσは両側の原子の種類(=点のラベル)により決まるものです。今回は以下のように明示的に与えてしまっています。

```
sigma_single = np.array([2.0 + 0.2 * x for x in range(8)])
epsilon_single = np.array([0.1 + 0.1 * x for x in range(8)])

self.sigma_matrix = 0.5 * (sigma_single[None, :] + sigma_single[:, None])
self.epsilon_matrix = np.sqrt(epsilon_single[None, :] * epsilon_single[:, None])

```

上の2行は同種原子の間でのεとσを決めています。別種原子の間でのεとσは、εについては相乗平均を、σについては相加平均をとっています。問題設定としては値が一意に決まっていればよいだけのものですが、このような別種原子のパラメータの決め方はLennard-JonesポテンシャルではLorentz-Berthelot rulesと呼ばれています。